# 0. Meta

## 0.1. Packages

In [48]:
import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, cross_val_predict

## 0.2. Functions

# 1. Data Import

In [3]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")

# 2. Model Training and Hyperparameter Tuning

## 2.1. Linear Regression

Train linear regression model and measure computation time.

In [44]:
lr_model = LinearRegression() 
start = datetime.now()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
stop = datetime.now()
delta = stop - start

Due to the log-transformation applied to the target variable y_train during data preprocessing to ensure normality and linearity, it is necessary to reverse this transformation on the predictions before assessing the model using metrics such as RMSE and R-squared.

In [38]:
y_test_log = np.log1p(y_test.to_numpy())

lr_pred_df = pd.DataFrame({'pred': lr_pred.flatten(), 'y_test': y_test_log.flatten()})

lr_pred_df['pred'] = np.expm1(lr_pred_df['pred'])
lr_pred_df['y_test'] = np.expm1(lr_pred_df['y_test'])

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in expm1
  result = getattr(ufunc, method)(*inputs, **kwargs)


During the back-transformation, NaNs and Infs occurred for some rows. These entries are eliminated, and the indices of the removed rows are stored to identify the observations that led to these issues. Additionally, the total count and the relative proportion of dropped rows are computed.

In [45]:
index_before = lr_pred_df.index
rows_before = lr_pred_df.shape[0]

lr_pred_df = lr_pred_df.replace([np.inf, -np.inf], np.nan).dropna()

index_after = lr_pred_df.index
rows_after = lr_pred_df.shape[0]

removed_indices = index_before.difference(index_after)
removed_rows = rows_before - rows_after
percent_removed = (removed_rows / rows_before) * 100

removed_rows_X_test = X_test.iloc[removed_indices]
removed_rows_y_test = y_test.iloc[removed_indices]
removed_rows_df = pd.concat([removed_rows_X_test, removed_rows_y_test], axis=1)
removed_rows_df['pre_backtrans_pred'] = lr_pred.flatten()[removed_indices]
filtered_removed_rows_df = removed_rows_df.loc[:, (removed_rows_df != 0).any()]

print('Rows before removing NaNs and Infs:', rows_before)
print('Rows after removing NaNs and Infs:', rows_after)
print('Number of rows removed:', removed_rows)
print('Percentage of rows removed:', percent_removed, '%')

print("Problematic rows in X_test and y_test:")
filtered_removed_rows_df

Rows before removing NaNs and Infs: 9193
Rows after removing NaNs and Infs: 9193
Number of rows removed: 0
Percentage of rows removed: 0.0 %
Problematic rows in X_test and y_test:


""


It appears that the linear regression model leads to extremely high predicted values for the problematic rows. Therefore, the back transformation fails, leading to infinite values. The full linear regression model therefore appears to be unsuitable. After removing the problematic rows, the evaluation metrics are calculated.  However, it should be noted that these can only be interpreted to a limited extent, as not all predictions are taken into account.

In [ ]:
print('Linear Regression\n')
lr_r2 = r2_score(lr_pred_df['y_test'], lr_pred_df['pred'])
print('R2: ', lr_r2)
lr_rmse = np.sqrt(mean_squared_error(lr_pred_df['y_test'], lr_pred_df['pred']))
print('Root Mean Squared Error: ', lr_rmse)
lr_seconds = delta.seconds + delta.microseconds/1E6
print('Time to compute: ', lr_seconds, 'seconds')

linear_reg = ('Linear Regression', lr_r2, lr_rmse, lr_seconds)

## 2.2. Regularized Linear Regression

### 2.2.1. Lasso Regression

### 2.2.2. Ridge Regression

### 2.2.3. Elastic Net Regression

## 2.3. Gaussian Process Regression

## 2.4. Bayesian Linear Regression

## 2.5. Robust Regression

### 2.5.1. Huber Regression

### 2.5.2. Quantile Regression

### 2.5.3. RANSAC Regression

### 2.5.4. Theil Sen Regression

## 2.6. K-Nearest Neighbors Regression

## 2.7. Artificial Neural Networks

### 2.7.1. Multi-Layer Perceptron Regressor

## 2.8. Support Vector Regression

## 2.9. Decision Trees Regression

## 2.10. Ensemble

### 2.10.1. Ada Boost Regressor

### 2.10.2. Bagging Regressor

### 2.10.3. Extra Tree Regressor

### 2.10.4. Gradient Boosting Regressor

### 2.10.5. XGBoost Regressor

### 2.10.6. LightGBM Regressor

### 2.10.7. Random Forest Regressor

### 2.10.8. Extra Trees Regressor

### 2.10.9. Stacking Regressor

### 2.10.10. Voting Regressor

### 2.10.11. Histogram-based Gradient Boosting Regressor

## 2.10. Dimensionality-Reduced Regression